1. Setup and Load Data

1.1. Install Dependencies and Setup

In [152]:
#making sure that we have the correct dependencies installed
%pip install tensorflow matplotlib

Note: you may need to restart the kernel to use updated packages.


In [153]:
#importing our dependencies
import tensorflow as tf
import os
#importing more 
import numpy as np
from matplotlib import pyplot as plt

1.2. Loading Data

In [154]:
#creating the data pipelines
training_data = tf.keras.utils.image_dataset_from_directory('Data/train', label_mode='categorical', shuffle=False) #generator
testing_data = tf.keras.utils.image_dataset_from_directory('Data/test', label_mode='categorical', shuffle=False)
validating_data = tf.keras.utils.image_dataset_from_directory('Data/valid', label_mode='categorical', shuffle=False)


Found 13492 files belonging to 100 classes.
Found 500 files belonging to 100 classes.
Found 500 files belonging to 100 classes.


2. Preprocessing Data by Scaling

In [155]:
#right now we have our batches, with values from 0-255. In order to make the learning more efficient, we want to scale down to 0-1

#the map function allows us to transform within the pipeline
#the x variable represents the images, y represents the class labels
#by doing x/255, we scale down to 0-1
training_data = training_data.map(lambda x, y: (x/255, y))
testing_data = testing_data.map(lambda x, y: (x/255, y))
validating_data = validating_data.map(lambda x, y: (x/255, y))

3. Deep Model

3.1. Building the Deep Learning Model

In [156]:
#importing more dependencies
from tensorflow.keras.layers import Input, Conv2D, SeparableConv2D, \
     Add, Dense, BatchNormalization, ReLU, MaxPool2D, GlobalAvgPool2D
from tensorflow.keras import Model 


In [157]:


def conv_bn(x, filters, kernel_size, strides=1):
    x = Conv2D(filters=filters,
               kernel_size=kernel_size,
               strides=strides,
               padding='same',
               use_bias=False)(x)
    x = BatchNormalization()(x)
    return x


def sep_bn(x, filters, kernel_size, strides=1):
    x = SeparableConv2D(filters=filters,
                        kernel_size=kernel_size,
                        strides=strides,
                        padding='same',
                        use_bias=False)(x)
    x = BatchNormalization()(x)
    return x


def entry_flow(x):
    x = conv_bn(x, filters=32, kernel_size=3, strides=2)
    x = ReLU()(x)
    x = conv_bn(x, filters=64, kernel_size=3)
    tensor = ReLU()(x)

    x = sep_bn(tensor, filters=128, kernel_size=3)
    x = ReLU()(x)
    x = sep_bn(x, filters=128, kernel_size=3)
    x = MaxPool2D(pool_size=3, strides=2, padding='same')(x)

    tensor = conv_bn(tensor, filters=128, kernel_size=1, strides=2)

    x = Add()([tensor, x])
    x = ReLU()(x)
    x = sep_bn(x, filters=256, kernel_size=3)
    x = ReLU()(x)
    x = sep_bn(x, filters=256, kernel_size=3)
    x = MaxPool2D(pool_size=3, strides=2, padding='same')(x)

    tensor = conv_bn(tensor, filters=256, kernel_size=1, strides=2)

    x = Add()([tensor, x])
    x = ReLU()(x)
    x = sep_bn(x, filters=728, kernel_size=3)
    x = ReLU()(x)
    x = sep_bn(x, filters=728, kernel_size=3)
    x = MaxPool2D(pool_size=3, strides=2, padding='same')(x)

    tensor = conv_bn(tensor, filters=728, kernel_size=1, strides=2)
    x = Add()([tensor, x])

    return x


def middle_flow(tensor):
    for _ in range(8):
        x = ReLU()(tensor)
        x = sep_bn(x, filters=728, kernel_size=3)
        x = ReLU()(x)
        x = sep_bn(x, filters=728, kernel_size=3)
        x = ReLU()(x)
        x = sep_bn(x, filters=728, kernel_size=3)

        tensor = Add()([tensor, x])

    return tensor


def exit_flow(tensor):
    x = ReLU()(tensor)
    x = sep_bn(x, filters=728, kernel_size=3)
    x = ReLU()(x)
    x = sep_bn(x, filters=1024, kernel_size=3)
    x = MaxPool2D(3, strides=2, padding='same')(x)

    tensor = conv_bn(tensor, filters=1024, kernel_size=1, strides=2)

    x = Add()([tensor, x])
    x = sep_bn(x, filters=1536, kernel_size=3)
    x = ReLU()(x)
    x = sep_bn(x, filters=2048, kernel_size=3)
    x = ReLU()(x)
    x = GlobalAvgPool2D()(x)
    x = Dense(units=100, activation='softmax')(x)

    return x



In [158]:
input = Input(shape=[256, 256, 3])

x = entry_flow(input)
x = middle_flow(x)
output = exit_flow(x)

model = Model(input, output)
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [159]:
hist = model.fit(training_data, epochs = 10, validation_data=validating_data)

Epoch 1/10
  5/422 ━━━━━━━━━━━━━━━━━━━━ 1:04:01 9s/step - accuracy: 0.4646 - loss: 3.8659

KeyboardInterrupt: 

3.2. Plotting Performance

In [160]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

NameError: name 'hist' is not defined

<Figure size 640x480 with 0 Axes>

In [161]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

NameError: name 'hist' is not defined

<Figure size 640x480 with 0 Axes>

4. Evaluation